## 참고자료
- 일부문자/tag포함한 xpath 찾기 : https://yeko90.tistory.com/entry/python-%EA%B8%B0%EC%B4%88-xpath%EB%A5%BC-%EC%9D%B4%EC%9A%A9%ED%95%98%EC%97%AC-%EC%9B%90%ED%95%98%EB%8A%94-%EC%A0%95%EB%B3%B4-%EC%B6%94%EC%B6%9C

# 라이브러리 import

In [1]:
import requests

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotVisibleException


from urllib.request import urlopen
import json
import re


import time
import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta

import pandas as pd 
import numpy as np
from tqdm import tqdm

# 맵리뷰페이지 접속

리뷰페이지 접속 방법
- 자신이 찾을 지점의 고유번호를 입력하기
- 고유번호 찾는 방법: 
    1. 네이버맵 사이트에서 [검색할 지점] 검색
    2. 바 형태로 뜬 지점 정보에 마우스 우클릭
    3. 프레임소스 보기 클릭
    4. 소스보기 화면 url에서 [지점 번호] 찾기
    ex. view-source:https://pcmap.place.naver.com/restaurant/[지점번호]/home?from=map&fromPanelNum=1&ts=1633655558076
    
- 리뷰페이지 url 예시 : https://pcmap.place.naver.com/restaurant/+[지점번호]+/home?from=map&amp;fromPanelNum=2&amp

## 매장명, 매장번호

## 매장명 받아오기 - 롯데리아
각 브랜드 홈페이지의 매장찾기에서 서울 전 지점명을 크롤링하기

In [2]:
chrome_path = 'C:/Users/chris/0 hyunji/Marketing/Crawling/' 
options = webdriver.ChromeOptions()
# options.add_argument('headless')#창 안 뜨게 설정
driver = webdriver.Chrome(chrome_path +'chromedriver.exe',options=options)# 크롬 경로
driver.implicitly_wait(3)

url = 'http://www.lotteria.com/Shop/Shop_List.asp#devCallShopList'
driver.get(url)
time.sleep(5)

KeyboardInterrupt: 

In [ ]:
html_cleaner = re.compile('<.*?>')

In [ ]:
store_lst = []
html= driver.page_source
soup = BeautifulSoup(html,'html.parser')
for s in soup.find_all('td',{'class':'first num'}):
    name = re.sub(html_cleaner,"",str(s))
    store_lst.append(name)

In [9]:
lotteria = pd.DataFrame(store_lst)
lotteria.columns= ['지점명']
lotteria.to_excel('롯데리아.xlsx', index= False)

다운타우너/브루클린/쉐이크쉑

In [5]:
store_list= pd.DataFrame()
store_list['지점명']= ['강남','청담','두타','센트럴시티','종각','영등포 타임스퀘어','용산 아이파크몰','노원','코엑스','홍대','신림']
store_list['지점번호']= ' '

## 매장 지점코드 받아오기

In [6]:
# store_list = pd.read_excel('./롯데리아.xlsx')
# store_list['지점번호']= ' '

In [10]:
chrome_path = 'C:/Users/chris/0 hyunji/Marketing/Crawling/' 
options = webdriver.ChromeOptions()
# options.add_argument('headless')#창 안 뜨게 설정
driver = webdriver.Chrome(chrome_path +'chromedriver.exe',options=options)# 크롬 경로
driver.implicitly_wait(3)

In [13]:
for i in store_list.loc[store_list['지점번호']==' ','지점명'].index:
    try: 
        url = 'https://maps.naver.com'
        driver.get(url)
        time.sleep(5)
        store_name = '쉐이크쉑 '+store_list.loc[i,'지점명']+'점'
#         store_name = store_list.loc[i,'지점명']

        driver.find_element_by_xpath('//input[contains(@autocomplete,"off")]').clear()
        driver.find_element_by_xpath('//input[contains(@autocomplete,"off")]').send_keys(store_name)
        driver.find_element_by_xpath('//input[contains(@autocomplete,"off")]').send_keys(Keys.ENTER)

        driver.switch_to.frame(driver.find_elements_by_xpath('//iframe[contains(@id,"searchIframe")]')[0])
        html = driver.page_source
#         driver.find_element_by_xpath(f'//a[contains(@class,"Ow5Yt")]'.format(store_name)).click()
        driver.find_element_by_xpath(f'//span[contains(text(),"{store_name}")]/../..'.format(store_name)).click()
        print(driver.current_url)
        store_number = re.findall('place\/'+'[0-9]+',driver.current_url)[0].split('/')[1]
        store_list.loc[i,'지점번호']= store_number
    except:
        print(store_name, i)
        pass
    time.sleep(np.random.uniform(3,5))


쉐이크쉑 강남점 0
https://map.naver.com/v5/search/%EC%89%90%EC%9D%B4%ED%81%AC%EC%89%91%20%EC%A2%85%EA%B0%81%EC%A0%90?c=14138676.6033362,4511130.9846631,15,0,0,0,dh
https://map.naver.com/v5/search/%EC%89%90%EC%9D%B4%ED%81%AC%EC%89%91%20%EC%98%81%EB%93%B1%ED%8F%AC%20%ED%83%80%EC%9E%84%EC%8A%A4%ED%80%98%EC%96%B4%EC%A0%90?c=14138676.6033362,4511130.9846631,15,0,0,0,dh
https://map.naver.com/v5/search/%EC%89%90%EC%9D%B4%ED%81%AC%EC%89%91%20%EC%BD%94%EC%97%91%EC%8A%A4%EC%A0%90?c=14138676.6033362,4511130.9846631,15,0,0,0,dh
https://map.naver.com/v5/search/%EC%89%90%EC%9D%B4%ED%81%AC%EC%89%91%20%EC%8B%A0%EB%A6%BC%EC%A0%90?c=14138676.6033362,4511130.9846631,15,0,0,0,dh


브루클린 더 버거 조인트

In [11]:
for i in store_list.loc[store_list['지점번호']==' ','지점명'].index:
    try:
        url = 'https://maps.naver.com'

        driver.get(url)
        time.sleep(5)
        store_name = '쉐이크쉑 '+store_list.loc[i,'지점명']+'점'
#         store_name = store_list.loc[i,'지점명']

        driver.find_element_by_xpath('//input[contains(@autocomplete,"off")]').clear()
        driver.find_element_by_xpath('//input[contains(@autocomplete,"off")]').send_keys(store_name)
        driver.find_element_by_xpath('//input[contains(@autocomplete,"off")]').send_keys(Keys.ENTER)

        driver.switch_to.frame(driver.find_elements_by_xpath('//iframe[contains(@id,"searchIframe")]')[0])
        # html = driver.page_source
        # driver.find_element_by_xpath(f'//a[contains(@class,"Ow5Yt")]'.format(store_name)).click()
        driver.find_element_by_xpath(f'//span[text()="{store_name}"]/../..'.format(store_name)).click()
        html = driver.page_source
        print(driver.current_url)
        store_number = re.findall('place\/'+'[0-9]+',driver.current_url)[0].split('/')[1]
        store_list.loc[i,'지점번호']= store_number
    except:
        print(store_name,i)
        pass
    time.sleep(np.random.uniform(3,5))

https://map.naver.com/v5/search/%EC%89%90%EC%9D%B4%ED%81%AC%EC%89%91%20%EA%B0%95%EB%82%A8%EC%A0%90?c=14139972.1729659,4509432.4074105,15,0,0,0,dh
쉐이크쉑 강남점 0
https://map.naver.com/v5/search/%EC%89%90%EC%9D%B4%ED%81%AC%EC%89%91%20%EC%B2%AD%EB%8B%B4%EC%A0%90/place/876646309?c=14138676.6033362,4511130.9846631,15,0,0,0,dh&placePath=%3Fentry%253Dbmp
https://map.naver.com/v5/search/%EC%89%90%EC%9D%B4%ED%81%AC%EC%89%91%20%EB%91%90%ED%83%80%EC%A0%90/place/855293917?c=14138676.6033362,4511130.9846631,15,0,0,0,dh&placePath=%3Fentry%253Dbmp
https://map.naver.com/v5/search/%EC%89%90%EC%9D%B4%ED%81%AC%EC%89%91%20%EC%84%BC%ED%8A%B8%EB%9F%B4%EC%8B%9C%ED%8B%B0%EC%A0%90/place/1553797864?c=14138676.6033362,4511130.9846631,15,0,0,0,dh&placePath=%3Fentry%253Dbmp
https://map.naver.com/v5/search/%EC%89%90%EC%9D%B4%ED%81%AC%EC%89%91%20%EC%A2%85%EA%B0%81%EC%A0%90?c=14138676.6033362,4511130.9846631,15,0,0,0,dh
쉐이크쉑 종각점 4
https://map.naver.com/v5/search/%EC%89%90%EC%9D%B4%ED%81%AC%EC%89%91%20%EC%98%81%EB%93%B1%E

다운타우너

In [217]:
# for i in store_list.loc[store_list['지점번호']==' ','지점명'].index:
#     try: 
#         url = 'https://maps.naver.com'
#         driver.get(url)
#         time.sleep(5)
#         #         store_name = '롯데리아 '+store_list.loc[i,'지점명']
#         store_name = store_list.loc[i,'지점명']

#         driver.find_element_by_xpath('//input[contains(@autocomplete,"off")]').clear()
#         driver.find_element_by_xpath('//input[contains(@autocomplete,"off")]').send_keys(store_name)
#         driver.find_element_by_xpath('//input[contains(@autocomplete,"off")]').send_keys(Keys.ENTER)

#         driver.switch_to.frame(driver.find_elements_by_xpath('//iframe[contains(@id,"searchIframe")]')[0])
#         html = driver.page_source
#         #         driver.find_element_by_xpath(f'//a[contains(@class,"Ow5Yt")]'.format(store_name)).click()
#         # driver.find_element_by_xpath(f'//span[contains(text(),"{store_name}")]/../..'.format(store_name)).click()
#         print(driver.current_url)
#         store_number = re.findall('place\/'+'[0-9]+',driver.current_url)[0].split('/')[1]
#         store_list.loc[i,'지점번호']= store_number
#     except:
#         print(store_name, i)
#         pass
#     time.sleep(np.random.uniform(3,5))


In [17]:
store_list.to_excel('./data/.xlsx',index=False)

## 매장명, 매장번호 데이터 불러오기

In [109]:
store_list = pd.read_excel('./data/롯데리아.xlsx')

In [110]:
store_list#.head()

,지점명,지점번호
0,신정네거리역,1428648745
1,청계산입구역,1024291405
2,홈플러스강동,1321912505
3,홈플러스월드컵,1154415775
4,용산역사,1517724323
...,...,...
174,언더랜드,11825111
175,서울랜드점,11825012
176,소공2호,11825032
177,동서울터미널,11824729


## 리뷰 탭에서 더보기 클릭

In [5]:
#홈에서의 더보기 경로는 홈페이지마다 위치가 달라지므로 f12 버튼 눌러서 파악해야 함
def review_seen(review_driver, see_more_xpath):
    #홈에서 스크롤 다운
    review_driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    time.sleep(np.random.uniform(0.5,2))
    #홈에서 방문자(블로그) 리뷰 더보기 클릭
    review_driver.find_element_by_xpath(see_more_xpath).click()

def all_visit_review_seen(review_driver):
    #방문자 리뷰에서 더보기 탭 끝까지 클릭하기 
    while True:
        try:
            review_driver.find_element_by_xpath('//*[@id="app-root"]/div/div/div[2]/div[5]/div[4]/div[4]/div[2]/a').click()
            review_driver.implicitly_wait(np.random.uniform(1,2))
        except:
            break

def all_blog_review_seen(review_driver):
    # 블로그 리뷰에서 더보기 탭 끝까지 클릭하기
    while True:
        try:
            review_driver.find_element_by_xpath('//*[@id="app-root"]/div/div/div[2]/div[5]/div[4]/div[2]/div[2]/a').click()
            review_driver.implicitly_wait(np.random.uniform(1,2))
        except:
            break

# 방문자 리뷰

## 필요한 정보 추출하기 - 정규표현식 
- 평균평점: 사용자가 다른 리뷰에서 준 평점 합산한 것의 평균
- 평점 : 사용자가 해당 장소에 대하여 평가한 평점 
- 방문횟수
- 날짜
- 리뷰(한줄평): 빈칸이면 'None'


**~cleaner: 특정 표현 제외** 

In [99]:
# html 태그 제거
html_cleaner = re.compile('<.*?>')
#평점 이외 표현 제거
score_cleaner = re.compile(r'\d{4}.\d{2}.\d{3}'+'번째 방문영수증')
hangul_cleaner = re.compile('[가-힣]+')


**~format : 특정표현 포함**


In [31]:
#평균 평점 추출용 : 평균평점+ 숫자
avg_score_format = re.compile('평균별점'+r'[0-9.]+')
#날짜 추출용: YYYY.MM.DD
# date_format = re.compile(r'\d{4}.\d{2}.\d{2}')
num_format = re.compile('[0-9\.]+')
#방문횟수 추출용: <n>번째 방문영수증 --> <n> 
visit_time_format = re.compile(r'\d'+'번째 방문') 

## take_info: 필요한 정보를 데이터프레임로 정리
- 리뷰, 방문횟수, 작성날짜, 평균평점, 평점

In [32]:
def take_info(soup):
    visitor_reviews_lst = [] #리뷰(한줄평)
    visit_times_lst = [] #방문횟수
    review_dates_lst = [] #작성날짜
    avg_score_lst = [] #평균평점
    score_lst = [] #평점

    for s in review_soup.find_all('li',{'class':'_3FaRE'}):

        visit_info = s.find('div',{'class':'_3-LAD'})

        if visit_info is None:
            continue
        visit_info = re.sub(html_cleaner,"",str(visit_info)) #html 태그 제거 
        info = re.findall(num_format, visit_info)
        if len(info) <3:
            #별점
            score_lst.append('None')
            #방문일자
            try: 
                rev_date = pd.to_datetime('2021.'+info[0], format = '%Y.%m.%d.')
            except:
                rev_date = pd.to_datetime('20'+info[0], format = '%Y.%m.%d.')
            review_dates_lst.append(rev_date)
            #방문횟수
            visit_times_lst.append(info[1])

        else:
            #별점
            score_lst.append(info[0])
            #방문일자
            try: 
                rev_date = pd.to_datetime('2021.'+info[1], format = '%Y.%m.%d.')
            except:
                rev_date = pd.to_datetime('20'+info[1], format = '%Y.%m.%d.')
            review_dates_lst.append(rev_date)
            #방문횟수
            visit_times_lst.append(info[2])
        ##-------------리뷰 추출
        try:
            review = s.find('span',{'class':'WoYOw'})
            review = re.sub(html_cleaner,"",str(review))
            visitor_reviews_lst.append(review)


        except:
            visitor_reviews_lst.append('None')
     
    #데이터 프레임으로 만들기 
    visit_reviews = pd.DataFrame({'평점':score_lst,'한줄평':visitor_reviews_lst,'작성날짜':review_dates_lst,'방문횟수':visit_times_lst})

    return visit_reviews

In [9]:
# def take_info(soup):
#     visitor_reviews_lst = [] #리뷰(한줄평)
#     visit_times_lst = [] #방문횟수
#     review_dates_lst = [] #작성날짜
#     avg_score_lst = [] #평균평점
#     score_lst = [] #평점


#     for s in soup.find_all('div',{'class':'_1Z_GL'}):

#         ##----------평균평점 추출
#         personal_info = s.find('div',{'class':'_23Rml'})
#         personal_info = re.sub(html_cleaner,"",str(personal_info))#html 태그 제거 

#         try:#평균평점에 소수점이 있는 경우
#             matchobj = avg_score_format.search(str(personal_info))
#             avg_score = matchobj.group()
#             avg_score_lst.append(avg_score[4:])

#         except:#평균평점이 정수인 경우
#             continue


#         ##-----------방문정보(평점, 날짜, 방문횟수) 추출
#         visit_info = s.find('div',{'class':'_1ZcDn'})
#         visit_info = re.sub(html_cleaner,"",str(visit_info)) #html 태그 제거 

#         #날짜 추출
#         matchobj = date_format.search(visit_info)
#         rev_date = matchobj.group()
#         review_dates_lst.append(rev_date)

#         #방문횟수 추출
#         matchobj = visit_time_format.search(visit_info)
#         visit_num = matchobj.group()
#         visit_times_lst.append(visit_num[0])

#         #평점 추출
#         score = re.sub(score_cleaner,"",visit_info)
#         score_lst.append(score)


#         ##-------------리뷰 추출
#         try:
#             review = s.find('div',{'class':'PVBo8'})
#             review = re.sub(html_cleaner,"",str(review))
#             visitor_reviews_lst.append(review)


#         except:
#             visitor_reviews_lst.append('None')
            
#     #데이터 프레임으로 만들기 
#     visit_reviews = pd.DataFrame({'평균평점':avg_score_lst,'평점':score_lst,'한줄평':visitor_reviews_lst,'작성날짜':review_dates_lst,'방문횟수':visit_times_lst})
        
#     return visit_reviews

## 전체 크롤링 코드

In [10]:
def visit_crawling(store_number,see_more_xpath):
    chrome_path = 'C:/Users/chris/0 hyunji/Marketing/Crawling/' 
    options = webdriver.ChromeOptions()
#     options.add_argument('headless')#창 안 뜨게 설정
    review_driver = webdriver.Chrome(chrome_path +'chromedriver.exe',options=options)# 크롬 경로
    review_driver.implicitly_wait(np.random.uniform(1.5,3))
    
    #url 접속
    review_url = 'https://pcmap.place.naver.com/restaurant/'+str(store_number)+'/home?from=map&amp;fromPanelNum=2&amp'
    review_driver.get(review_url)
    time.sleep(np.random.uniform(2,5))

    # store_name= []
    main_html = review_driver.page_source
    main_soup = BeautifulSoup(main_html,'html.parser')
    store_name=main_soup.find('span', {'class': '_3XamX'}) #이름
    store_name=re.sub(html_cleaner, "", str(store_name))
    #     store_name.append(name)

    #스크롤 다운 후 방문자 리뷰 더보기 클릭
    try:
        review_seen(review_driver, visit_xpath)
    except:
        review_seen(review_driver, visit_xpath)

    # #방문자 리뷰 더보기 끝까지 클릭
    all_visit_review_seen(review_driver)
    review_html = review_driver.page_source
    review_soup = BeautifulSoup(review_html,'html.parser')

    #review_df에 필요한 정보 담기
    review_df = take_info(review_soup)
    review_df.insert(0,'지점명확인',store_name,allow_duplicates=False)
    review_driver.close()
    return review_df

# 블로그 리뷰

## blog_link_list, blog_review_content 
- 블로그 리뷰 링크 받기
- 블로그 내의 내용 받기

In [100]:
def blog_link_list(blog_soup):
    global html_cleaner, hangul_cleaner
    blog_url_lst=[]
    date_lst= []
    before_1y = datetime.datetime.now() - timedelta(days = 365)
    for s in blog_soup.find_all('li',{'class':'_3OyGD'}):
        date = s.find('span',{'class':'_2M6JC'})
        date = re.sub(html_cleaner,"", str(date))
        if '일 전' in date:
            date = datetime.datetime.now() - timedelta(days = int(date[0]))
        elif '시간 전' in date:
            date = datetime.datetime.now()
        else:
            date = re.sub(hangul_cleaner,"", str(date))
            try:
                date = pd.to_datetime(date,format='%m.%d.')+relativedelta(years = 121)

            except:
                continue #1년 전 글 
    #             date = pd.to_datetime('20'+date,format='%Y.%m.%d.')        
        url = s.find('a')['href']
        blog_url_lst.append(url)
        date_lst.append(date) 

    blog_df = pd.DataFrame({'url':blog_url_lst, 'date':date_lst})
    blog_df['content'] = ' '
    blog_df = blog_df.loc[blog_df['url'].str.contains('blog'),:]
    return blog_df

In [12]:
# # 3년 이내의 블로그 리뷰만 링크 받기
# def blog_link_list(blog_soup):
#     global html_cleaner
#     blog_url_lst=[]
#     date_lst= []
#     before_1y = datetime.datetime.now() - timedelta(days = 365)
#     for s in blog_soup.find_all('li',{'class':'_3OyGD'}):
#         date = s.find('span',{'class':'_7OvgA'})
#         date = re.sub(html_cleaner,"", str(date))
#         if '일 전' in date:
#             date = datetime.datetime.now() - timedelta(days = int(date[0]))
#         elif '시간 전' in date:
#             date = datetime.datetime.now()

#         #1년 이내 블로그 글만 퍼오기
#         if pd.to_datetime(date) > before_1y:
#             url = s.find('a')['href']
#             blog_url_lst.append(url)
#             date_lst.append(date)            
#     blog_df = pd.DataFrame({'url':blog_url_lst, 'date':date_lst})
#     blog_df['content'] = ' '
#     blog_df = blog_df.loc[blog_df['url'].str.contains('blog'),:]
#     return blog_df

In [13]:
def blog_review_content(blog_driver,blog_df):
    
    for url in tqdm(blog_df['url']):
        blog_driver.get(url)
        time.sleep(5)
        blog_driver.switch_to.frame('mainFrame')
        content = ''
        overlays = ".se-component.se-text.se-l-default" # 내용 크롤링
        unrefined_content = blog_driver.find_elements_by_css_selector(overlays)
        
        if len(unrefined_content) == 0:
            overlays = ".se_component.se_paragraph.default" # 내용 크롤링
            unrefined_content = blog_driver.find_elements_by_css_selector(overlays)
        for c in unrefined_content:
            content = content + c.text # content 라는 값에 + 하면서 점점 누적
            content = content.replace("\n","")
        blog_df.loc[blog_df['url']==url,'content'] = content

    return blog_df

## 전체 크롤링 코드

In [14]:
def blog_crawling(store_number,see_more_xpath):
    chrome_path = 'C:/Users/chris/0 hyunji/Marketing/Crawling/' 
    options = webdriver.ChromeOptions()
#     options.add_argument('headless')#창 안 뜨게 설정
    blog_driver = webdriver.Chrome(chrome_path+'chromedriver.exe',options=options)# 크롬 경로
    blog_driver.implicitly_wait(np.random.uniform(1.5,3))
    
    #url 접속
    review_url = 'https://pcmap.place.naver.com/restaurant/'+str(store_number)+'/home?from=map&amp;fromPanelNum=2&amp'
    blog_driver.get(review_url)
    time.sleep(5)
    

    # 블로그 리뷰 탭으로 넘어가기
    review_seen(blog_driver, see_more_xpath)
    # 더보기 탭 누르기
    all_blog_review_seen(blog_driver)
    # 현재 html 받기
    blog_html = blog_driver.page_source
    blog_soup = BeautifulSoup(blog_html,'html.parser')
    html_cleaner = re.compile('<.*?>')

    #블로그 링크, 날짜 --> 블로그 df에 저장
    blog_df = blog_link_list(blog_soup)
    #블로그 내용 크롤링 --> 블로그 df에 저장
    blog_df = blog_review_content(blog_driver, blog_df)
    blog_df.insert(0,'지점명',store_name_lst[i],allow_duplicates=False)
    
    blog_driver.close()
    return blog_df

# 크롤링 실행 후 df에 저장

## 크롤링한 후 전체 df로 합치기

In [156]:
visit_xpath ='//span[contains(text(),"방문자 리뷰")]/..'
blog_xpath= '//span[contains(text(),"블로그리뷰")]/..'

In [111]:
store_number_lst = store_list['지점번호']
store_name_lst = store_list['지점명']

## 주문자 리뷰

In [112]:
visit_review_df = pd.read_excel('./롯데리아_방문자리뷰.xlsx')

In [114]:
visit_review_df.tail()

,지점명,지점명확인,평균평점,평점,한줄평,작성날짜,방문횟수
42860,개봉역,롯데리아 개봉역점,4.8,4,치킨버거 + 클래식치즈버거 1+1 행사 하길래 구입했네요^^*,2019.12.23,4
42861,개봉역,롯데리아 개봉역점,3.5,4,뭐 개봉역이랑 가까워서 자주 들러요~,2019.12.15,1
42862,개봉역,롯데리아 개봉역점,4.8,4,새우버거 1+1 리아데이 행사라서 구매 했네요,2019.12.05,3
42863,개봉역,롯데리아 개봉역점,4.4,5,직원이 친절해요,2019.12.03,1
42864,개봉역,롯데리아 개봉역점,4.0,4,메뉴가 빠르게 나와서 좋네요!,2019.12.03,1


In [164]:
# visit_review_df = pd.read_excel('./롯데리아_방문자리뷰.xlsx')
for i in tqdm(range(178,len(store_number_lst))):    
    #i번째 지점의 크롤링 review_df 생성
    try:
        visit_xpath ='//span[contains(text(),"방문자 리뷰")]/..'
        review_df = visit_crawling(store_number_lst[i],visit_xpath)
    except NoSuchElementException:
        visit_xpath ='//span[contains(text(),"주문자 리뷰")]/..'
        review_df = visit_crawling(store_number_lst[i],visit_xpath)
        
    #review_df의 첫 열에 지점명 추가
    review_df.insert(0,'지점명',store_name_lst[i],allow_duplicates=False)
    #전체 df에 합치기
    visit_review_df = pd.concat([visit_review_df, review_df],axis=0)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [01:27<00:00, 87.55s/it]


In [166]:
visit_review_df.to_excel('./롯데리아_방문자리뷰.xlsx',index=False)

In [165]:
visit_review_df

,지점명,지점명확인,평균평점,평점,한줄평,작성날짜,방문횟수
0,이대,롯데리아 이대점,4.9,5,굿,2021.09.26,2
1,이대,롯데리아 이대점,4.9,5,냠냠,2021.09.26,1
2,이대,롯데리아 이대점,4.8,5,좋아요,2021.09.26,1
3,이대,롯데리아 이대점,4.0,4,맛있어요,2021.09.24,2
4,이대,롯데리아 이대점,4.5,5,좋음,2021.09.14,2
...,...,...,...,...,...,...,...
595,봉천,롯데리아 서울대입구역점,NaN,5,롯데리아는 불고기버거죠,2019-03-24 00:00:00,1
596,봉천,롯데리아 서울대입구역점,NaN,3,사람이 늘 많아요^^,2019-03-17 00:00:00,1
597,봉천,롯데리아 서울대입구역점,NaN,4,24시간 오픈해서 좋습니다.\n배달도 되요,2019-03-04 00:00:00,1
598,봉천,롯데리아 서울대입구역점,NaN,5,깔끔한 내부 그리고 맛있는 버거와 커피 : ),2019-02-28 00:00:00,1


In [170]:
store_list.loc[store_list['지점명']=='미아삼거리',]

,지점명,지점번호
116,미아삼거리,20981518


In [163]:
store_list.iloc[178,:]

지점명           봉천
지점번호    36893807
Name: 178, dtype: object

## 블로그 리뷰

In [168]:
blog_review_df = pd.read_excel('./롯데리아_블로그리뷰.xlsx')

In [ ]:
# blog_review_df = pd.read_excel('./롯데리아_블로그리뷰')
for i in tqdm(store_list.loc[store_list['지점명'].isin(remain_lst),:].index):
    #i번째 지점의 크롤링 review_df 생성
    try:
        review_df = blog_crawling(store_number_lst[i],blog_xpath)
        #전체 df에 합치기
        blog_review_df = pd.concat([blog_review_df, review_df],axis=0)
    except:
        continue

 15%|████████████▊                                                                      | 8/52 [01:52<10:58, 14.96s/it]Exception ignored in: <function Service.__del__ at 0x000002340F6B8DC0>
Traceback (most recent call last):
  File "C:\Anaconda3\lib\site-packages\selenium\webdriver\common\service.py", line 176, in __del__
    self.stop()
  File "C:\Anaconda3\lib\site-packages\selenium\webdriver\common\service.py", line 151, in stop
    self.send_remote_shutdown_command()
  File "C:\Anaconda3\lib\site-packages\selenium\webdriver\common\service.py", line 132, in send_remote_shutdown_command
    if not self.is_connectable():
  File "C:\Anaconda3\lib\site-packages\selenium\webdriver\common\service.py", line 115, in is_connectable
    return utils.is_connectable(self.port)
  File "C:\Anaconda3\lib\site-packages\selenium\webdriver\common\utils.py", line 106, in is_connectable
    socket_ = socket.create_connection((host, port), 1)
  File "C:\Anaconda3\lib\socket.py", line 796, in create_conn

In [173]:
blog_review_df.to_excel('롯데리아_블로그리뷰.xlsx', index=False)

In [176]:
remain_lst = np.setdiff1d(store_list['지점명'],blog_review_df['지점명'])

In [179]:
remain_lst

array(['GS마트상계', '강남우성사거리', '강변씨네마', '개포', '건대스타시티', '구의역', '내린천휴게소',
       '녹번', '답십리', '동대문광장', '동서울터미널', '롯데마트시흥배곧', '망우', '목동킴스클럽',
       '문정로데오', '번동DT', '사가정역', '사당', '삼양사거리', '상봉역', '상암', '서울랜드2호',
       '서울랜드점', '석계', '세곡', '수서역사', '시흥중앙', '시흥휴게소', '신김포공항', '신방화역',
       '암사역', '언더랜드', '역삼', '영등포타임스퀘어B2', '오금', '오류동역', '용산역사', '용산역사ST',
       '잠실새내', '정릉', '중앙대', '창동역', '창천', '천호역', '청량리', '학동역', '홈플러스남현',
       '홈플러스상봉', '홈플러스신내', '홈플러스월곡', '홈플러스월드컵', '홈플러스합정'], dtype=object)

In [181]:
store_list.loc[store_list['지점명'].isin(remain_lst),:].index

Int64Index([  3,   4,  13,  17,  19,  20,  21,  26,  33,  37,  42,  43,  54,
             55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,
             73,  88,  93, 100, 101, 102, 104, 106, 110, 111, 112, 114, 117,
            121, 126, 128, 131, 139, 148, 149, 156, 161, 169, 174, 175, 177],
           dtype='int64')

## 디버깅

### 방문자

In [137]:
i = 164
chrome_path = 'C:/Users/chris/0 hyunji/Marketing/Crawling/' 
options = webdriver.ChromeOptions()
# options.add_argument('headless')#창 안 뜨게 설정
review_driver = webdriver.Chrome(chrome_path +'chromedriver.exe',options=options)# 크롬 경로
review_driver.implicitly_wait(3)

#url 접속
review_url = 'https://pcmap.place.naver.com/restaurant/'+str(store_number_lst[i])+'/home?from=map&amp;fromPanelNum=2&amp'
review_driver.get(review_url)
time.sleep(5)

# store_name= []
main_html = review_driver.page_source
main_soup = BeautifulSoup(main_html,'html.parser')
store_name=main_soup.find('span', {'class': '_3XamX'}) #이름
store_name=re.sub(html_cleaner, "", str(store_name))
#     store_name.append(name)

#스크롤 다운 후 방문자 리뷰 더보기 클릭
try:
    review_seen(review_driver, visit_xpath)
    
except:
#     review_driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    review_seen(review_driver, visit_xpath)
    
# #방문자 리뷰 더보기 끝까지 클릭
all_visit_review_seen(review_driver)
review_html = review_driver.page_source
review_soup = BeautifulSoup(review_html,'html.parser')



In [138]:
#review_df에 필요한 정보 담기
review_df = take_info(review_soup)
review_df.insert(0,'지점명확인',store_name,allow_duplicates=False)
review_df.insert(0,'지점명',store_name_lst[i],allow_duplicates=False)

In [139]:
visit_review_df = pd.concat([visit_review_df,review_df], axis=0)

In [140]:
visit_review_df.tail()

,지점명,지점명확인,평균평점,평점,한줄평,작성날짜,방문횟수
595,면목중앙,롯데리아 면목중앙점,NaN,5,롯데리아는 불고기버거죠,2019-03-24 00:00:00,1
596,면목중앙,롯데리아 면목중앙점,NaN,3,사람이 늘 많아요^^,2019-03-17 00:00:00,1
597,면목중앙,롯데리아 면목중앙점,NaN,4,24시간 오픈해서 좋습니다.\n배달도 되요,2019-03-04 00:00:00,1
598,면목중앙,롯데리아 면목중앙점,NaN,5,깔끔한 내부 그리고 맛있는 버거와 커피 : ),2019-02-28 00:00:00,1
599,면목중앙,롯데리아 면목중앙점,NaN,4,학생들이 시끄럽고 서비스는 평타,2019-02-23 00:00:00,1


In [ ]:
pd.set_option('display.max_rows',8000)
visit_review_df

### 블로그

In [90]:
chrome_path = 'C:/Users/chris/0 hyunji/Marketing/Crawling/' 
options = webdriver.ChromeOptions()
# options.add_argument('headless')#창 안 뜨게 설정
blog_driver = webdriver.Chrome(chrome_path+'chromedriver.exe',options=options)# 크롬 경로
blog_driver.implicitly_wait(5)

#url 접속
review_url = 'https://pcmap.place.naver.com/restaurant/'+str(store_number_lst[0])+'/home?from=map&amp;fromPanelNum=2&amp'
blog_driver.get(review_url)
time.sleep(5)


# 블로그 리뷰 탭으로 넘어가기
review_seen(blog_driver, blog_xpath)
# 5번 더보기 탭 누르기
all_blog_review_seen(blog_driver)
# 현재 html 받기
blog_html = blog_driver.page_source
blog_soup = BeautifulSoup(blog_html,'html.parser')
html_cleaner = re.compile('<.*?>')


# #블로그 링크, 날짜 --> 블로그 df에 저장
# blog_df = blog_link_list(blog_soup)
# #블로그 내용 크롤링 --> 블로그 df에 저장
# blog_df = blog_review_content(blog_driver, blog_df)
# blog_df.insert(0,'지점명',store_name_lst[0],allow_duplicates=False)



In [97]:
blog_url_lst=[]
date_lst= []
before_1y = datetime.datetime.now() - timedelta(days = 365)
for s in blog_soup.find_all('li',{'class':'_3OyGD'}):
    date = s.find('span',{'class':'_2M6JC'})
    date = re.sub(html_cleaner,"", str(date))

    if '일 전' in date:
        date = datetime.datetime.now() - timedelta(days = int(date[0]))
    elif '시간 전' in date:
        date = datetime.datetime.now()
    else:
        date = re.sub(hangul_cleaner,"", str(date))
        try:
            date = pd.to_datetime(date,format='%m.%d.')+relativedelta(years = 121)

        except:
            continue #1년 전 글 
#             date = pd.to_datetime('20'+date,format='%Y.%m.%d.')        
    url = s.find('a')['href']
    blog_url_lst.append(url)
    date_lst.append(date) 

blog_df = pd.DataFrame({'url':blog_url_lst, 'date':date_lst})
blog_df['content'] = ' '
blog_df = blog_df.loc[blog_df['url'].str.contains('blog'),:]

In [91]:
#블로그 링크, 날짜 --> 블로그 df에 저장
blog_df = blog_link_list(blog_soup)
#블로그 내용 크롤링 --> 블로그 df에 저장
blog_df = blog_review_content(blog_driver, blog_df)
blog_df.insert(0,'지점명',store_name_lst[0],allow_duplicates=False)


NameError: name 'hangul_cleaner' is not defined